Authors: Xinyan Yang, Fangjun Zhu

Date: 12/11/2019

# 1 Introduction

## Collaborative filtering system

Typically, the workflow of a collaborative filtering movie recommendation system is:
1. Users express their preferences towards different types of movies by rating a series of films on a scale of 1-5 with 5 being the most favorable rating.<br>
2. The system matches this user's ratings against other users' and finds the people with most "similar" tastes of movies.<br>
3. With similar users, the system recommends movies that the similar users have rated highly but not yet being rated by this user.<sup>[1]

## Goals

1. Build a collaborative filtering movie recommendation system.<br>
2. Develop a python GUI application by Tkinter to recommend movies for users via their ratings towards some randomly selected movies. 

# 2 Recommendation System

## Setup

In [1]:
import math
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.linalg as la
import time
import random

## Prepare data

The movie rating dataset, ml-latest-small, is collected from MovieLens https://grouplens.org/datasets/movielens/. This dataset shows a 5-star rating and tagging activity of a group of users. It includes 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Here, only the movies and ratings are of interest. There are some basic rules in the data preparation process:
* The original rating data is read to a $N$ x $P$ matrix $X$. It is the rating activity of $P$ users with respect to $N$ movies. Assume there are $K$ fundamental factors that each user's preference towards the movies can be explained by. 
*  The data is split into ten folds. Nine folds are for training and the remaining one fold is for testing.
* Default value 0 is given to any entry without a rating in matrix $X$.

### Fetch the data

Download data from MovieLens website and create dataframes for movies and ratings.

In [2]:
# save movies data to dataframe
movies = pd.read_csv("ml-latest-small/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# save ratings data to dataframe
ratings = pd.read_csv("ml-latest-small/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### Process the data

Transform the movies and ratings dataframes into 2D arrays (rating matrix). Then split the dataset into two sets, one for training and the other for testing.

In [4]:
def split(movies, ratings, k_fold):
    """Return training and testing data as 2D arrays by splitting the rating matrix created from the dataframes,
       as well as a list of movie names.
 
    movies -- dataframe of movies
    ratings -- dataframe of ratings
    """
    # each column in the rating matrix is one-user ratings towards different movies
    # each row in the rating matrix is one-movie ratings by different users 
    # since movieIDs are from a much larger movie pool, they are not continuous. 
    # here use continuous indices of movies dataframe to represent movies. 
    col = ratings['userId'].apply(lambda x: x-1).values
    new_movideID = ratings['movieId'].unique()
    row = ratings['movieId'].apply(lambda x: np.where(new_movideID == x)[0][0]).values
    data = ratings['rating'].values

    # randomly split and get index sets for training and testing set.
    seq = np.arange(len(col))
    np.random.shuffle(seq)
    idx_train = seq[:len(seq)*(k_fold-1)//k_fold]
    idx_test = seq[len(seq)*(k_fold-1)//k_fold:]
    # training
    col_train = col[idx_train]
    row_train = row[idx_train]
    data_train = data[idx_train]
    # testing
    col_test = col[idx_test]
    row_test = row[idx_test]
    data_test = data[idx_test]

    # create sparse matrix
    X_train = sp.coo_matrix((data_train,(row_train, col_train)), 
                            shape=(len(new_movideID), len(ratings['userId'].unique()))).toarray()
    X_test = sp.coo_matrix((data_test,(row_test, col_test)), 
                           shape=(len(new_movideID), len(ratings['userId'].unique()))).toarray()

    temp = movies['movieId'].values
    movie_name = [movies.iloc[np.where(temp==mID)[0][0]]['title'] for mID in new_movideID]
    
    return X_train, X_test, movie_name

In [5]:
X_train, X_test, movie_name = split(movies, ratings, 10)
print('The shape of rating matrix with training data: {0}.'.format(np.shape(X_train)))
print('The shape of rating matrix with testing data: {0}.'.format(np.shape(X_test))), 
print(movie_name[:4])

The shape of rating matrix with training data: (9724, 610).
The shape of rating matrix with testing data: (9724, 610).
['Toy Story (1995)', 'Grumpier Old Men (1995)', 'Heat (1995)', 'Seven (a.k.a. Se7en) (1995)']


## Complete rating matrix 

The matrix from the ml-latest-small dataset is sparsely populated, since an individual user has only rated a few of the movies available. Completing the rating maxtrix is equivalent to determining $C$ and $W$ such that:

$$CW{\approx}X$$

The squares cost function of this matrix factorization problem with $l_2$-regularizers:

$$\sum_{(i,j)\in \Omega}(c^iw_j - x_{ij})^2 + \lambda ||C||_2^2 + \lambda ||W||_2^2$$

First, define an error calculation function.

In [6]:
def error(X, C, W):
    """Return the mean squared error of the rating matrix X and its estimated values.
 
    X -- N x P observed rating matrix (2D array)
    C -- N x K matrix (2D array) from matrix factorization
    W -- K x P matrix (2D array) from matrix factorization
    """
    flag = (X > 0)
    flag[flag == True] = 1
    flag[flag == False] = 0
    flag = flag.astype(np.float64)
    MSE = np.mean((flag * (X - np.matmul(C, W)))**2)
    
    return MSE

Complete the matrix by alternatively optimizing the cost function.

In [7]:
def matrix_complete(X_train, X_test, k, lam=10, maxiter=50, count_time=True, print_iter=True):
    """Return the estimated rating matrix from the observed sparsely populated rating matrix X.
 
    X_train -- N x P observed rating matrix
    X_test -- N x P observed testing matrix
    k -- the assumed numnber of fundamental factors
    lam -- the parameter for the two l2-regularizers
    maxiter -- maximum number of iterations
    count_time -- (boolean) decide to print elapsed time or not
    """
    
    X = X_train.copy()
    t_start = time.time()
    
    # flag entires with no rating values
    flag = (X > 0)
    flag[flag == True] = 1
    flag[flag == False] = 0
    flag = flag.astype(np.float64)

    # get matrix dimension info
    N = X.shape[0]
    P = X.shape[1]

    # initialize C and W
    W_init = np.ones((k, P))
    C_init = np.ones((N, k))
    C = C_init.copy()
    W = W_init.copy()

    for iter in range(maxiter):

        # enumerate every row in flag matrix (2D array) to update W
        for i, flag_i in enumerate(flag.T):
            # set C value to zero if there is no rating
            zero_idx = np.where(flag_i==0)[0]
            C_zero = C.copy()
            C_zero[zero_idx, :]=0        
            L = np.linalg.cholesky( np.matmul(C_zero.T, C) + np.diag(lam*np.ones(k)))
            temp = np.linalg.solve(L, np.matmul(C_zero.T, X[:,i]))
            W[:,i] = np.linalg.solve(L.T, temp)

        # enumerate every col in flag matrix (2D array) to update C
        for j, flag_j in enumerate(flag):
            # set WT value to zero if there is no rating
            zero_idx = np.where(flag_j==0)[0]
            W_zero = W.copy()
            W_zero[:,zero_idx]=0
            L = np.linalg.cholesky( np.matmul(W_zero, W.T) + np.diag(lam*np.ones(k)) )
            temp = np.linalg.solve(L, np.matmul(W_zero, X[j,:]))
            C[j] = np.linalg.solve(L.T, temp)  
    
        if (iter+1)%10 == 0 and print_iter:
            print('Iteration {0}: Training MSE: {1:.4f}, Testing MSE: {2:.4f}'
                  .format(iter+1, error(X_train, C, W), error(X_test, C, W)))
    
    if count_time:
        print('Elapsed time: {0:.2f} s'.format(time.time() - t_start))

    return C, W

Train the model.

In [8]:
C, W = matrix_complete(X_train, X_test, 5, lam=10, maxiter=50, count_time=True, print_iter=True)

Iteration 10: Training MSE: 0.0143, Testing MSE: 0.0024
Iteration 20: Training MSE: 0.0100, Testing MSE: 0.0024
Iteration 30: Training MSE: 0.0100, Testing MSE: 0.0024
Iteration 40: Training MSE: 0.0100, Testing MSE: 0.0024
Iteration 50: Training MSE: 0.0100, Testing MSE: 0.0024
Elapsed time: 43.36 s


## Make recommendations 

Movie recommendations are made to each user based on the highest value of the user's estimated rating activity over the movies he/she has not watched.

In [9]:
def recommend(X, C, W, movie_name):
    """ Return a list of movies. The ith element in the list is the movie recommended for the ith user.
    
    X -- N x P observed rating matrix used for training
    C -- N x K matrix (2D array) from matrix factorization from trained model
    W -- P x K matrix (2D array) from matrix factorization from trained model
    movie_name -- a list of movie names
    """
    # estimated rating matrix
    X_est = np.matmul(C, W)
    
    # define a masked array of X_est to prevent recommending movies users already watched
    watched = np.zeros(X.shape, dtype=bool)
    watched[np.where(X != 0)] = True  # spot the rated movies
    X_masked = np.ma.masked_array(X_est, mask=watched)
    
    # list of recommendations
    movie_list_rec = [movie_name[i] for i in X_masked.argmax(axis=0)]

    return movie_list_rec

Ready to make recommendations.

In [10]:
movie_list_rec = recommend(X_train, C, W, movie_name)
print('The movie recommended for the first 10 users: {0}'.format(movie_list_rec[0:10]))

The movie recommended for the first 10 users: ['Patton (1970)', 'Pulp Fiction (1994)', 'Pulp Fiction (1994)', 'Patton (1970)', 'Silence of the Lambs, The (1991)', 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)', 'Shawshank Redemption, The (1994)', 'Matrix, The (1999)', 'Pulp Fiction (1994)', 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)']


# 3 Application

Here, a python GUI application is developed by Tkinter. The user is first asked to rate a list of randomly selected movies. Then the input rating activity will be appended to the $X$ rating matrix obtained before from the MovieLens dataset. With the new $X$ matrix, the model will be trained again. After training, the last movie in the movie_list_rec will be the recommended movie for the new user. Finally, the app will display the movie which is tailored to the user's taste. Relevant codes are in the Appendix.

The Youtube video inclues a short demo of this app: https://youtu.be/anGgSRu01nY

# 4 References

[1] Wikipedia contributors. (2019, October 31). Collaborative filtering. In Wikipedia, The Free Encyclopedia. Retrieved 22:54, December 1, 2019, from https://en.wikipedia.org/w/index.php?title=Collaborative_filtering&oldid=923933501 <br>

[2] Jeremy Watt, Reza Borhani, Aggelos K. Katsaggelos, Machine Learning Refined: Foundations, Algorithms, and Applications, Cambridge University Press, New York, NY, 2016

# Appendix

Codes used to develop the python GUI.

https://www.datacamp.com/community/tutorials/gui-tkinter-python#ITT

In [11]:
import os
import re
from collections import OrderedDict
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import cv2
from threading import Thread
from google_images_download import google_images_download as google_image

In [12]:
class MyVideoCapture:
    
    def __init__(self, video_source=0):
        # Open the video source
        self.vid = cv2.VideoCapture(video_source)
        if not self.vid.isOpened():
            raise ValueError("Unable to open video source", video_source)
        # Get video source width and height
        self.width = self.vid.get(cv2.CAP_PROP_FRAME_WIDTH)
        self.height = self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT)

    def get_frame(self, loop=True):
        if self.vid.isOpened():
            ret, frame = self.vid.read()
            if ret:
                # Return a boolean success flag and the current frame converted to BGR
                return (ret, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            else:
                if loop:
                    self.vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
                    ret, frame = self.vid.read()
                    return (ret, frame)
                
                return (ret, None)
        else:
            return (False, None)

    # Release the video source when the object is destroyed
    def __del__(self):
        if self.vid.isOpened():
            self.vid.release()

In [13]:
class VideoCanvas(tk.Canvas):
    
    def __init__(self, master, framerate=25, **kwargs):
        super(VideoCanvas, self).__init__(master, **kwargs)
        self._playing = False
        self._src = None
        self._framerate = framerate
        self._frame = None

    @property
    def is_playing(self):
        return self._playing

    def set_source(self, path):
        self._src = MyVideoCapture(path)

    def play(self):
        if not self._src:
            raise ValueError('Call set_source() first')
        self._playing = True
        self._update()

    def stop(self):
        self._playing = False
        self._src = None

    def _update(self):
        if not self._playing:
            return

        ok, frame = self._src.get_frame()
        if ok:
            self._frame = ImageTk.PhotoImage(image=Image.fromarray(frame))
            self.create_image(0, 0, image=self._frame, anchor=tk.NW)
            # Call this function again after a short delay
            self.master.after(1000 // self._framerate, self._update)
        else:
            self._playing = False

In [14]:
class MovieGui(tk.Frame):
    
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        
        self.movie_db = pd.read_csv("ml-latest-small/movies.csv")
        self.ratings_db = pd.read_csv("ml-latest-small/ratings.csv")

        # Divides into a left and right panel
        self._master_panels = tk.PanedWindow(parent)
        self._master_panels.pack(fill=tk.BOTH, expand=True)

        # LEFT PANEL: rate movies
        self._left_frame = tk.Frame(parent)
        self._left_frame.pack(fill=tk.BOTH, expand=True)
        self._master_panels.add(self._left_frame, width=500)

        # Scrollable area (only canvas is scrollable)
        self._rating_canvas = tk.Canvas(self._left_frame)
        self._rating_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self._rating_scroll = tk.Scrollbar(self._left_frame, orient=tk.VERTICAL, command=self._rating_canvas.yview)
        self._rating_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        self._rating_canvas.configure(yscrollcommand=self._rating_scroll.set)

        # Container for the movies to rate
        self._movie_frame = tk.Frame(self._rating_canvas)
        # tk.Button(movie_frame, Update, command=update_ratings_list).pack()
        self._movie_frame.pack(fill=tk.BOTH, expand=True)
        self._movie_frame.columnconfigure(2, weight=1)
        tk.Label(parent,
                 text='Please rate movies you have seen on a scale from 1-5 (0 if you have not seen the movie).\n'
                      'When you are done, hit the "Recommend" button on the right!').pack(fill=tk.BOTH, expand=True)
        self._ratings_data = OrderedDict()

        # Create the movie rows
        for idx, (mid, movie) in enumerate(get_random_movies(self.movie_db)):
            # Movie image
            data = {}
            img_path = get_image(movie)
            if img_path:
                img = Image.open(img_path).resize((60, 80))
                img = ImageTk.PhotoImage(img)
                # Images must be stored somewhere or they will be garbage collected
                img_lbl = tk.Label(self._movie_frame, image=img)
                img_lbl._movie_image = img
                img_lbl.grid(row=idx, column=0)

            # Movie title
            tk.Label(self._movie_frame, text=movie).grid(row=idx, column=1)

            # Slider and number input
            rating = tk.StringVar(0)
            data['rating'] = rating
            tk.Scale(self._movie_frame, from_=0, to=5, orient=tk.HORIZONTAL, variable=rating, showvalue=False)\
            .grid(row=idx, column=2)
            tk.Spinbox(self._movie_frame, from_=0, to=5, textvariable=rating, width=3).grid(row=idx, column=3)

            data['id'] = mid
            self._ratings_data[movie] = data

        self._rating_canvas.create_window((0, 0), window=self._movie_frame, anchor=tk.NW)

        # RIGHT PANEL: top: get recommendation; bottom: show movie playing
        self._right_panels = tk.PanedWindow(parent, orient=tk.VERTICAL)
        self._master_panels.add(self._right_panels)

        self._rec_frame = tk.Frame(parent)
        self._rec_frame.pack(fill=tk.BOTH, expand=True)
        self._recommendation_img_lbl = tk.Label(self._rec_frame)
        self._recommendation_img_lbl.pack(fill=tk.BOTH, expand=True)
        self._recommendation_lbl = tk.Label(self._rec_frame)
        self._recommendation_lbl.pack(fill=tk.X, expand=True)
        self._rec_button = tk.Button(self._rec_frame, text='What should I watch?', command=self._get_recommendation)
        self._rec_button.pack()
        self._progress = ttk.Progressbar(self._rec_frame, orient=tk.HORIZONTAL, mode='indeterminate')
        self._progress.pack(fill=tk.X, expand=True)
        ttk.Separator(self._rec_frame).pack(fill=tk.X, expand=True)
        self._right_panels.add(self._rec_frame, height=400)

        self._video_frame = tk.Frame(self._right_panels)
        self._video_frame.pack(fill=tk.BOTH, expand=True)
        self._video_canvas = VideoCanvas(self._video_frame, width=300, height=200)
        self._video_canvas.pack(fill=tk.BOTH, expand=True)
        self._play_button = tk.Button(self._video_frame, text='Play/Pause', command=self.toggle_video)
        self._play_button.pack(side=tk.LEFT)
        self._right_panels.add(self._video_frame, height=200)

    def play_video(self, source='Christmas.mp4'):
        self._video_canvas.set_source(source)
        self._video_canvas.play()

    def toggle_video(self):
        if self._video_canvas.is_playing:
            self._video_canvas.stop()
        else:
            self.play_video()

    def get_user_ratings(self):
        movies = []
        ratings = []
        for movie, data in self._ratings_data.items():
            rating = int(data['rating'].get())
            ratings.append(rating)
            movies.append(movie)
        return (movies, ratings)

    def _update_recommendation(self, movie):
        path = get_image(movie)
        img = ImageTk.PhotoImage(Image.open(path).resize((200, 300)))
        self._recommendation_img_lbl._movie_img = img
        self._recommendation_img_lbl.configure(image=img)
        self._recommendation_lbl.configure(text=movie)
        self._rec_button.configure(state=tk.NORMAL)
        self._progress.stop()

    def _get_recommendation(self):
        self._rec_button.config(state=tk.DISABLED)
        self._progress.start()
        self._worker = Thread(target=self._do_matrix_complete)
        self._worker.start()
        
    def _do_matrix_complete(self):
        user_id = max(self.ratings_db['userId']) + 1
        movies, ratings = self.get_user_ratings()
        ratings = [float(r) for r in ratings]
        movie_ids = [self._ratings_data[m]['id'] for m in movies]
        
        new_entry = {'userId': [user_id] * len(movies), 
                     'movieId': movie_ids, 
                     'rating': ratings, 
                     'timestamp': [0] * len(movies)}
        df = pd.DataFrame(data=new_entry)
        
        new_ratings = self.ratings_db.append(df, ignore_index=True)
        X_train_new, X_test_new, movie_name = split(self.movie_db, new_ratings, 10)
        C_new, W_new = matrix_complete(X_train_new, X_test_new, 5, lam=5, maxiter=20, print_iter=False)
        recommendations = recommend(X_train_new, C_new, W_new, movie_name)
        
        rec = random.choice(recommendations)
        print('New recommendation: ' + str(rec))
        
        # Call this on the GUI thread
        self.master.after(0, self._update_recommendation, rec)

In [15]:
def get_random_movies(movies, num=8):
    indices = np.random.randint(0, len(movies), num)
    ret = []
    for m in indices:
        name = movies.loc[m][1]
        mid = movies.loc[m][0]
        ret.append((mid, name))
    return ret

In [16]:
def get_image(movie):
    
    path = None
    key = ''.join(filter(lambda c: re.match('[a-zA-Z0-9 _-]', c), movie))
    cache_root = './downloads/' + key
    if os.path.exists(cache_root):
        try:
            paths = os.listdir(cache_root)
            path = cache_root + '/' + next(iter(paths))
        except StopIteration:
            path = None

    if not path:
        res = google_image.googleimagesdownload()
        paths = res.download(dict(limit=1, keywords=key))
        if paths:
            path = paths[0][key]
            if path and os.path.exists(path[0]):
                path = path[0]

    print(path)
    return path

In [17]:
def clear_image_cache():
    import shutil
    try:
        shutil.rmtree('./downloads')
    except FileNotFoundError:
        pass

Let's find the best movie for you!

In [18]:
clear_image_cache()
root = tk.Toplevel()
root.geometry('1200x800')
root.title('Movie Recommendation')
gui = MovieGui(root)
root.mainloop()


Item no.: 1 --> Item name = Beneath the Planet of the Apes 1970
Evaluating...
Starting Download...
Completed Image ====> 1.MV5BMjljYjY0MDktZjdlNC00ZTAxLTkzM2ItM2JjZjQ3YzQ0NzJiL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_.jpg

Errors: 0

/Users/aaple/PycharmProjects/Fangjun_final_project/downloads/Beneath the Planet of the Apes 1970/1.MV5BMjljYjY0MDktZjdlNC00ZTAxLTkzM2ItM2JjZjQ3YzQ0NzJiL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_.jpg

Item no.: 1 --> Item name = The Last Witch Hunter 2015
Evaluating...
Starting Download...
Completed Image ====> 1.MV5BMjM5Njk5MzYzM15BMl5BanBnXkFtZTgwNzM1Mjk4NjE@._V1_.jpg

Errors: 0

/Users/aaple/PycharmProjects/Fangjun_final_project/downloads/The Last Witch Hunter 2015/1.MV5BMjM5Njk5MzYzM15BMl5BanBnXkFtZTgwNzM1Mjk4NjE@._V1_.jpg

Item no.: 1 --> Item name = Buffalo Soldiers 2001
Evaluating...
Starting Download...
Completed Image ====> 1.Buffalo_Soldiers_film_poster.jpg

Errors: 0

/Users/aaple/PycharmProjects/Fangjun_final_project/downloads/Buff

/Users/aaple/anaconda3/lib/python3.7/site-packages/PIL/Image.py:2731: DecompressionBombWarning: Image size (100419400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


Elapsed time: 44.84 s
New recommendation: Inside Job (2010)

Item no.: 1 --> Item name = Inside Job 2010
Evaluating...
Starting Download...
Completed Image ====> 1.MV5BMTQ3MjkyODA2Nl5BMl5BanBnXkFtZTcwNzQxMTU4Mw@@._V1_.jpg

Errors: 0

/Users/aaple/PycharmProjects/Fangjun_final_project/downloads/Inside Job 2010/1.MV5BMTQ3MjkyODA2Nl5BMl5BanBnXkFtZTcwNzQxMTU4Mw@@._V1_.jpg
